In [ ]:
#general imports. Add as I see fit.
import numpy as np
import pandas as pd

In [2]:
#General function for opening .txt files, which it seems like all the inputs are.
def get_input(filepath):
    f = open(filepath, "r")
    arr = f.read().split("\n")
    arr.pop()
    return arr

In [ ]:
##NOTE - cells below that are not commented are works in progress/assorted trials.
##       Will work to comment in cells where it does work

In [ ]:
##SOLUTION CODE FOR 12/8 (part 1).

df = pd.read_csv('input_file_12_8.txt', sep=' ', header=None)
df.columns = ['Command', 'Iterator']

df['Visited'] = 0

def cycle_through(frame, accum, row=None):
    if not row:
        x = 0
    else:
        x = row
    if frame.loc[x, 'Visited'] == 1:
        return accum
    else:
        if frame.loc[x, 'Command'] == 'nop':
            frame.loc[x, 'Visited'] = 1
            return cycle_through(frame, accum, x+1)
        elif frame.loc[x, 'Command'] == 'acc':
            y = accum + frame.loc[x, 'Iterator']
            frame.loc[x, 'Visited'] = 1
            return cycle_through(frame, y, x+1)
        else:
            frame.loc[x, 'Visited'] = 1
            new_row = frame.loc[x, 'Iterator']
            return cycle_through(frame, accum, x + new_row)

#This will print correct answer for part 1, verified on website
print(cycle_through(df, 0))

df['Visited'] = 0

df['End Binary'] = 0
df.iloc[-1, 3] = 1

def replace(command):
    if command == 'nop':
        return 'jmp'
    elif command == 'jmp':
        return 'nop'
    else:
        return 'acc'
    
def cycle_through_new(frame, accum, row=None):
    if not row:
        x = 0
    else:
        x = row
        
    if frame.loc[x, 'End Binary'] == 1:
        return (accum, True)
    
    if frame.loc[x, 'Visited'] == 1:
        return (accum, False)
    else:
        #Copy here just to be safe
        new_frame = frame.copy()
        if frame.loc[x, 'Command'] == 'nop':
            new_frame.loc[x, 'Visited'] = 1
            return cycle_through_new(new_frame, accum, x+1)
        elif frame.loc[x, 'Command'] == 'acc':
            y = accum + frame.loc[x, 'Iterator']
            new_frame.loc[x, 'Visited'] = 1
            return cycle_through_new(new_frame, y, x+1)
        else:
            new_frame.loc[x, 'Visited'] = 1
            new_row = frame.loc[x, 'Iterator']
            return cycle_through_new(new_frame, accum, x + new_row)
        
for i in range(626):
    #Had to do this because simply assigning it to be the same would cause changes to propogate backwards
    #So annoying.
    df_new = df.copy()
    #print(df_new.iloc[i])
    df_new.iloc[i, 0] = replace(df_new.iloc[i, 0])
    #print(df_new.iloc[i])
    val = cycle_through_new(df_new, 0)
    if val[1] == True:
        print(val[0])
        break

In [3]:
##SOLUTION CODE FOR 12_9 Part 1
#Gets solution for part 1
lst = get_input('input_file_12_9.txt')
lst = [int(i) for i in lst]

for i in range(26, len(lst)):
    temp_lst = lst.copy()[i-26:i-1]
    x = lst[i-1]
    break_bool = False
    #print(temp_lst)
    #print(x)
    for j in range(0, 24):
        for k in range(j+1, 25):
            if temp_lst[j] + temp_lst[k] == x:
                break_bool = True
            if break_bool:
                continue
        if break_bool:
            continue
    if not break_bool:
        print(x)         
print('Done')

##SOLUTION CODE FOR 12_9 Part 1
#Gets solution for part 1
lst = get_input('input_file_12_9.txt')
lst = [int(i) for i in lst]

solution = 90433990

def dec_9_function(lst, solution):
    for i in range(len(lst)):
        running_sum = 0
        counter = 0
        while running_sum < solution:
            running_sum = sum(lst[i:counter])
            if running_sum == solution:
                return max(lst[i:counter]) + min(lst[i:counter])
            counter += 1

print(dec_9_function(lst, 90433990))

90433990
Done


11691646
